In [38]:
%load_ext autoreload
%autoreload 2

0it [00:00, ?it/s]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dependencies

In [39]:
import datasets_handler
import evaluation_metrics
from zeroberto import ZeroBERTo
import gc
import pandas as pd
from datasets import load_dataset, Dataset


In [40]:
# df = pd.read_csv('/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_clean_df_articles.csv',nrows=4000)
# df.to_csv('/Users/alealcoforado/Documents/Projetos/Datasets/folhauol/folhauol_4000examples_clean_df_articles.csv')

In [41]:
which_dataset = 'folhauol' 
max_inferences = 2500

hyp_template = "Esse texto está relacionado a {}."

raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
classes_list = list(raw_data[class_col].unique())

In [56]:
zeroshot_method = "dotproduct"  

zeroshot_config = {
    'similarity_model' : 'sentence-transformers/stsb-xlm-r-multilingual',
    'setfit_model' : 'sentence-transformers/stsb-xlm-r-multilingual',# 'ricardo-filho/bert-base-portuguese-cased-nli-assin-2'
    'dataset':which_dataset,
    'class_col':class_col,
    'data_col':data_col,
    'split':"zeroshot",
    'labeling_method':zeroshot_method,
    'max_inferences':max_inferences,
    'classes':classes_list,
    'template': hyp_template,
    'random_state':422,
    'top_n': 8,
    'training_examples': 8,
    "batch_size" : 4,
    "num_pairs" : 5,
    "num_epochs" : 1,
    'st_train_epochs': 10,
    'st_train_batch_size': 20,
    # Unfreeze the head and freeze the body -> head-only training
    'keep_body_frozen_setfit': False,
    # Unfreeze the head and unfreeze the body -> end-to-end training
    # zeroshot_config['keep_body_frozen_setfit'] = False
}

data_to_label = raw_data.sample(zeroshot_config['max_inferences'],random_state=zeroshot_config['random_state']).sort_index()


In [57]:
model = ZeroBERTo(classes_list=zeroshot_config['classes'],labeling_dataset=data_to_label,
                embeddingModel=zeroshot_config['similarity_model'],config=zeroshot_config)
       

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


1
queries
3
4


In [58]:
model.runLabeling()
model.evaluateLabeling()

model.getLabelingMetrics()
model.saveLabelingResults()


Preds: 500  - Total time: 46.12 seconds - ETA: 3.8 minutes
Preds: 1000  - Total time: 87.79 seconds - ETA: 3.7 minutes
Preds: 1500  - Total time: 126.06 seconds - ETA: 3.5 minutes
Preds: 2000  - Total time: 163.55 seconds - ETA: 3.4 minutes
Preds: 2500  - Total time: 199.29 seconds - ETA: 3.3 minutes
top 1: {'accuracy': 0.5}
top 2: {'accuracy': 0.5833333333333334}
top 3: {'accuracy': 0.6111111111111112}
top 4: {'accuracy': 0.625}
top 5: {'accuracy': 0.5666666666666667}
top 6: {'accuracy': 0.5277777777777778}
top 7: {'accuracy': 0.5119047619047619}
top 8: {'accuracy': 0.5208333333333334}
top 9: {'accuracy': 0.49074074074074076}
top 10: {'accuracy': 0.49166666666666664}
top 11: {'accuracy': 0.4772727272727273}
top 12: {'accuracy': 0.4722222222222222}
top 13: {'accuracy': 0.47435897435897434}
top 14: {'accuracy': 0.47619047619047616}
top 15: {'accuracy': 0.4722222222222222}
top 16: {'accuracy': 0.4635416666666667}
top 2500: {'accuracy': 0.3312}
predictions_and_probabilities_test_2023_04_0

In [43]:
model.labeling_metrics


{'weighted': [{'accuracy': 0.3312},
  {'precision': 0.47446351511668045},
  {'recall': 0.3312},
  {'f1': 0.2857409631254401}],
 'macro': [{'accuracy': 0.3312},
  {'precision': 0.29801013834612505},
  {'recall': 0.33676202779769193},
  {'f1': 0.249588354980113}]}

In [44]:
### load results from labeling step and create training data for contrastive learning
zeroshot_previous_data = datasets_handler.getZeroshotPreviousData(
    model.config['dataset'],model.config['class_col'],top_n=model.config['top_n'],exec_time=model.config['exec_time'])

In [49]:
df_train = datasets_handler.splitDataset(model.labeling_dataset,model.config) ### df_test will not be used
train_dataset = datasets_handler.buildDatasetDict(df_train)
test_dataset = datasets_handler.buildDatasetDict(model.labeling_dataset) # test will not be used

/Users/alealcoforado/Documents/Projetos/ZeroBERTo/zeroberto/datasets_handler.py:165: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  train_dataset = Dataset.from_dict(df_train[['text','class_code']].to_dict('list'))


In [ ]:
# raw_data_final, model.config['new_class_col'] = datasets_handler.mergeLabelingToDataset(
#     raw_data,zeroshot_previous_data,model.config['class_col'])
# df_train['']
train_data = datasets_handler.buildDatasetDict(df_train)

In [59]:
### overrides config for looping second part of the model
# model.config['num_epochs'] = 5
# model.config['num_pairs'] = 20
# model.config['keep_body_frozen_setfit'] = False

### build trainer for contrastive learning
model.buildTrainer(train_dataset)

### train
model.contrastive_train()
gc.collect()

### predict labeling dataset with new classifier
model.getPredictions()
# 5000 sentences --- 4min40s ---sts-xlm-rr
# 5000 sentences ---  ---ricardo-filho/bert-portuguese

setfit_all_metrics = evaluation_metrics.get_metrics(model.y_pred, model.trainer.eval_dataset["class_code"])
print(setfit_all_metrics)

setfit_exec_time  = evaluation_metrics.saveResults(model.config,setfit_all_metrics)
print(model.config)

Applying column mapping to training dataset
***** Running training *****
  Num examples = 960
  Num epochs = 1
  Total optimization steps = 120
  Total train batch size = 8


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/120 [00:00<?, ?it/s]

In [ ]:
model.y_pred

NameError: name 'model' is not defined